In [1]:
import pandas

import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn

# Variable declarations
FILE_PATH = "Filtered_Data.csv"
SEPARATOR = ","
CLICK_THRESHOLD = 1000
MAX_ROWS = 500
MISCELLANEOUS = 'misc'
FEATURE_NAME = ['ad_advertiser_id', 'analyzer_name', 'click_browser', 'enriched_derived_device', 'enriched_derived_os', 'raw_sync_partner_id']

# imported the CSV data
dataframe = pandas.read_csv(FILE_PATH, sep = SEPARATOR)

# Drop the unused column
if 'Unnamed: 0' in dataframe.columns:
    dataframe.drop('Unnamed: 0', axis = 1, inplace = True)

# Remove the same prefix from all columns names
dataframe.columns  = [ column_name.split('.')[1] for column_name in dataframe.columns ]

CLICK_THRESHOLD = max(1000, int(float(dataframe.shape[0]) * .01))

In [2]:
# removing all data having single values in entire columns
for column in dataframe.columns:
    if(dataframe[column].nunique() == 1):
        dataframe.drop(column, axis = 1, inplace = True)

In [3]:
# Remove the timestamp data and impression system date data - results not dependent on the time frame
dataframe.drop('ts', axis = 1, inplace = True)
dataframe.drop('impression_system_date', axis = 1, inplace = True)

# Server hour do not matters
dataframe.drop('hour', axis = 1, inplace = True)

# Data have only 3 values of *au* region, and conversion status 0 for all of them
dataframe.drop('enriched_country', axis = 1, inplace = True)

# Data is similar to cvr_base_data_table.click_city
dataframe.drop('impression_city', axis = 1, inplace = True)

# Ad-Bid is the cost which is irrespecti   ve of conversion from user side
dataframe.drop('ad_bid', axis = 1, inplace = True)

# Ad CPA goal do not affects the conversion status
dataframe.drop('ad_cpa_goal', axis = 1, inplace = True)

# Click Conf GMT Offset do not affects the conversion status
dataframe.drop('click_conf_gmt_offset', axis = 1, inplace = True)

# Date Time do not affects and is too granuler to looku[, thus do not affects the conversion status
dataframe.drop('click_user_datetime', axis = 1, inplace = True)

# Publisher Domain do not affects the conversion status
# dataframe.drop('enriched_publisher_domain', axis = 1, inplace = True)

# Impression ISP Name do not affects the conversion status
dataframe.drop('impression_isp_name', axis = 1, inplace = True)

# below data is too granular to predict something
dataframe.drop('ad_keyword', axis = 1, inplace = True)
dataframe.drop('click_city', axis = 1, inplace = True)
dataframe.drop('raw_sub_publisher_id', axis = 1, inplace = True)
dataframe.drop('category_name', axis = 1, inplace = True)
dataframe.drop('click_id', axis = 1, inplace = True)
dataframe.drop('cc_id', axis = 1, inplace = True)
dataframe.drop('raw_publisher_id', axis = 1, inplace = True)
dataframe.drop('raw_sub_sub_publisher_id', axis = 1, inplace = True)

In [4]:
# # Following column names have less entries thus can be relabelled
# total = dataframe.shape[0]
# pandas.set_option('display.max_rows', MAX_ROWS)

# # Function for aggregation on basis of click, conversion and CVR(conversion ratio)
# func = {'conversion_status' : {
#     'conversion' : 'sum',
#     'click' : 'count'
# }}

# # Iterate through above columns list and 'misc' the data which is too sparse to use 
# # for classification or modelling
# for col in FEATURE_NAME:
#     dfg = dataframe.groupby(col).agg(func).reset_index()
#     dfg.columns = dfg.columns.droplevel(0)
#     dfg['cvr'] = dfg['conversion'] / dfg['click']
#     dfg['click percentage of total data'] = dfg['click'] / total * 100
#     declareMisc = set()
#     for index, row in dfg.iterrows():
#         if(row['click'] >= CLICK_THRESHOLD):
#             continue
#         declareMisc.add(row[''])
#     print (declareMisc)
#     for vals in declareMisc:
#         dataframe.loc[dataframe[col] == vals, col] = MISCELLANEOUS
    
#     print (dataframe[col].value_counts(), '\n\n')

In [ ]:
func = {'conversion_status' : {
    'conversion' : 'sum'
    'click' : 'count'
}}

eps = float(0.001)
def equals(d1, d2):
    if(len(d1) != len(d2)):
        return False
    d1_keys = d1.keys()
    d2_keys = d2.keys()
    for key in d1_keys:
        if(key in d2):
            if(abs(float(d1[key]) - float(d2[key])) > eps):
                return False
        else:
            return False
    return True

original_data = {}
last_values = {}
cur_values = {}

# compute data for initalizing cur_values dict
dfg = dataframe.groupby(['enriched_publisher_domain']).agg(func).reset_index()


while equals(last_values, cur_values):
    